# Spectral Methods Simulations

This notebook is dedicated to understanding decomposition via spectral (eigen) methods. This was created right after Min Cheol's epiphany that PCA is essentially same as spectral embedding with linear kernels (i.e., dot product) as affinity matrix. 

I will attempt to connect PCA/tICA using very intuitive spectral decompositions, and essentially make analogies to spectral embedding with various kernels.

In [188]:
import numpy as np
import numpy.linalg as la
from sklearn.preprocessing import normalize
from sklearn.datasets import make_spd_matrix

## PCA computation via eigendecomposition of the covariance matrix vs the Gram matrix

Let $X$ be 
Stereotypical PCA involves computing the eigendecomposition of the empirical covariance matrix, or $\frac{1}{N}X^{T}X$. 

Let us say that $u_{i}$ is an eigenvector of the covariance matrix, such that $\frac{1}{N}X^{T}Xu_{i} = \lambda_{i}u_{i}$. Multiplying by $X$ on the right on both sides yields:

$$\frac{1}{N}XX^{T}Xu_{i} = X\lambda_{i}u_{i}$$
$$\frac{1}{N}(XX^{T})(Xu_{i}) = \lambda_{i}(Xu_{i})$$

We can see that $Xu_{i}$ is an eigenvector of $XX^T$. However, this will not be normalized (have L2 norm of 1). Once $Xu_i$ is normalized to have L2 norm of 1, it will be equivalent to the eigenvectors of $XX^T$.

We can confirm that the eigenvectors displayed at the bottom-most cell (computed from the Gram matrix) are equal to the principal components computed from the eigenvectors of the covariance matrix.

In [207]:
# Generate some random data with each column centered at 0
X = np.concatenate([make_spd_matrix(3), make_spd_matrix(3)])
mean = X.mean(axis=0)
X = X - mean
sigma = X.T.dot(X)
pca_ev, pca_dirs = la.eigh(sigma)
print(pca_ev)
print('There are 3 nonzero eigenvalues')

[  0.10309278   9.15561158  13.38297902]
There are 3 nonzero eigenvalues


In [209]:
print('Eigenvectors:')
print(pca_dirs)

Eigenvectors:
[[ 0.7421334  -0.65364654  0.1482708 ]
 [ 0.12805063 -0.07887106 -0.98862652]
 [ 0.65790657  0.75267892  0.025167  ]]


In [210]:
normalize(X.dot(pca_dirs), axis=0, norm='l2')

array([[-0.379745  , -0.43295877,  0.50538303],
       [ 0.06517576,  0.42376967, -0.36933134],
       [-0.0967069 ,  0.44113556, -0.1108904 ],
       [ 0.85070409, -0.30838822,  0.08085442],
       [-0.32447016, -0.47143067, -0.59325252],
       [-0.11495779,  0.34787243,  0.4872368 ]])

In [214]:
pca_ev, pca_comps = la.eigh(X.dot(X.T))
pca_comps[:, -3:]

array([[ 0.379745  ,  0.43295877, -0.50538303],
       [-0.06517576, -0.42376967,  0.36933134],
       [ 0.0967069 , -0.44113556,  0.1108904 ],
       [-0.85070409,  0.30838822, -0.08085442],
       [ 0.32447016,  0.47143067,  0.59325252],
       [ 0.11495779, -0.34787243, -0.4872368 ]])

## tICA computation via the time lagged correlation matrix and the covariance matrix

In [61]:
dX = np.zeros((5,3))
dX[1: :] = X[:-1, :]

In [62]:
dX

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.15301319, -0.26339432,  0.11038113],
       [-0.34623484,  0.18708211,  0.15915272],
       [-0.07768646,  0.31951736, -0.2418309 ],
       [-0.21377436,  0.20959988,  0.00417447]])

In [64]:
X.T

array([[ 0.15301319, -0.34623484, -0.07768646, -0.21377436, -0.13261876],
       [-0.26339432,  0.18708211,  0.31951736,  0.20959988, -0.26065998],
       [ 0.11038113,  0.15915272, -0.2418309 ,  0.00417447,  0.39327875]])

In [215]:
la.inv(sigma)

array([[ 5.39070004,  0.91647514,  4.68261123],
       [ 0.91647514,  0.23276174,  0.80883686],
       [ 4.68261123,  0.80883686,  4.26048332]])

In [216]:
la.matrix_rank(sigma)

3